In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="gpt-oss:20b", temperature=0)

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

system_prompt = """You are a travel agent. 
IMPORTANT: When using the search-flight tool, you MUST use the following rules:
1. Dates must be in 'dd/mm/yyyy' format (e.g., 31/03/2026).
2. All dates must be in the future.
3. If the user asks for a one-way flight, ensure you still provide the correct parameters required by the tool.
No follow up questions."""

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt=system_prompt
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Get me a direct flight from San Francisco to Tokyo on March 31st")]},
    config
    )

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Get me a direct flight from San Francisco to Tokyo on March 31st', additional_kwargs={}, response_metadata={}, id='cde57d70-4273-4372-9329-93da0ef5e485'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2026-02-25T21:02:31.048714603Z', 'done': True, 'done_reason': 'stop', 'total_duration': 439037480052, 'load_duration': 57934694918, 'prompt_eval_count': 1066, 'prompt_eval_duration': 106222279115, 'eval_count': 1092, 'eval_duration': 273318213333, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--019c9695-9786-7f31-ac7a-1583fd1bdf65-0', tool_calls=[{'name': 'search-flight', 'args': {'cabinClass': 'M', 'curr': 'USD', 'departureDate': '31/03/2026', 'departureDateFlexRange': 0, 'flyFrom': 'San Francisco', 'flyTo': 'Tokyo', 'locale': 'en', 'passengers': {'adults': 1}, 'returnDate': '31/03/2026', 'returnDateFlexRange': 0, 'sort': 'price'}, 'id': 'bd94b

In [7]:
print(response["messages"][-1].content)

I’m sorry, but there are no direct flights from San Francisco to Tokyo on 31 March 2026.  
You might consider:

- **Flights with one layover** (e.g., via Los Angeles, Seattle, or a major Asian hub).  
- **Adjusting the travel date** by a day or two to catch a direct service.  
- **Checking alternative airports** such as Oakland (OAK) or San Jose (SJC) for better options.

**Fun fact:** Tokyo’s Shibuya Crossing is the world’s busiest pedestrian scramble, with up to 3,000 people crossing at once during peak hours! Have a great trip!


In [8]:
response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Get the flight from Los Angeles")]},
    config
    )

In [9]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Get me a direct flight from San Francisco to Tokyo on March 31st', additional_kwargs={}, response_metadata={}, id='cde57d70-4273-4372-9329-93da0ef5e485'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2026-02-25T21:02:31.048714603Z', 'done': True, 'done_reason': 'stop', 'total_duration': 439037480052, 'load_duration': 57934694918, 'prompt_eval_count': 1066, 'prompt_eval_duration': 106222279115, 'eval_count': 1092, 'eval_duration': 273318213333, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--019c9695-9786-7f31-ac7a-1583fd1bdf65-0', tool_calls=[{'name': 'search-flight', 'args': {'cabinClass': 'M', 'curr': 'USD', 'departureDate': '31/03/2026', 'departureDateFlexRange': 0, 'flyFrom': 'San Francisco', 'flyTo': 'Tokyo', 'locale': 'en', 'passengers': {'adults': 1}, 'returnDate': '31/03/2026', 'returnDateFlexRange': 0, 'sort': 'price'}, 'id': 'bd94b

In [10]:
print(response["messages"][-1].content)

I couldn’t find any flights from Los Angeles to Tokyo on 31 March 2026.  
You might want to try a different date, a nearby airport (e.g., LAX, OAK, or SJC), or a route with a layover.  

Let me know if you’d like me to search again with a different option!


In [11]:
response = await agent.ainvoke(
    {"messages": [HumanMessage(content="""Get the flight from San Francisco or 
                                        Los Angeles with at most one layover""")]},
    config
    )

In [13]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Get me a direct flight from San Francisco to Tokyo on March 31st', additional_kwargs={}, response_metadata={}, id='cde57d70-4273-4372-9329-93da0ef5e485'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2026-02-25T21:02:31.048714603Z', 'done': True, 'done_reason': 'stop', 'total_duration': 439037480052, 'load_duration': 57934694918, 'prompt_eval_count': 1066, 'prompt_eval_duration': 106222279115, 'eval_count': 1092, 'eval_duration': 273318213333, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--019c9695-9786-7f31-ac7a-1583fd1bdf65-0', tool_calls=[{'name': 'search-flight', 'args': {'cabinClass': 'M', 'curr': 'USD', 'departureDate': '31/03/2026', 'departureDateFlexRange': 0, 'flyFrom': 'San Francisco', 'flyTo': 'Tokyo', 'locale': 'en', 'passengers': {'adults': 1}, 'returnDate': '31/03/2026', 'returnDateFlexRange': 0, 'sort': 'price'}, 'id': 'bd94b

In [12]:
print(response["messages"][-1].content)

I’m sorry, but there are no flights from either San Francisco or Los Angeles to Tokyo on 31 March 2026 that fit the “at most one layover” criteria.  
It’s possible that the airline schedule for that exact day is very limited, or that the Kiwi API simply doesn’t have data for that future date.

**What you can do next:**

- Try a different travel date (e.g., 30 March or 1 April) – many airlines add more flights a few days before or after a specific date.
- Check nearby airports (e.g., Oakland (OAK) or San Jose (SJC) for LA‑area flights).
- Look for flights with a single layover in a major hub (e.g., Los Angeles → Tokyo via Seattle or San Francisco → Tokyo via Seattle).

**Fun fact:** Tokyo’s Shibuya Crossing is the world’s busiest pedestrian scramble, with up to 3,000 people crossing at once during peak hours! Have a great trip!


In [ ]:
response = await agent.ainvoke(
    {"messages": [HumanMessage(content="""Get the first three flights from San Francisco 
                                            or Los Angeles with at most one layover""")]},
    config
    )

In [ ]:
from pprint import pprint

pprint(response)

In [ ]:
print(response["messages"][-1].content)